# Exploded Pie

In this example we'll create a pie plot with one of the slices exploded, showing its subcategories on the side.

## Data loading

We'll use the NASA meteorite dataset. To do so we'll need first to run the data manager using the %run magic code.

In [ ]:
%run data_manager.py

At this poing we can load the data directly into a pandas dataframe

In [ ]:
df = load_meteorites()
df.head()

## Selecting the plotted data
Three columns are good candidates for a pie chart:

In [ ]:
print(df.nametype.unique())
print(df.fall.unique())
print(df.recclass.unique())

Given that we'd like to keep things simple, we are going to use `nametype` and `fall`. Let's extract the data.

In [ ]:
#counting nametypes
nametypes_counts = df.nametype.value_counts(normalize = True)
print(nametypes_counts)

In [ ]:
#counting fall status for "valid" asteroids
fall_counts_valid = df[df['nametype'] == 'Valid'].fall.value_counts(normalize = True)
print(fall_counts_valid)

## First implementation, by hand

A first implementation, step by step

In [ ]:
#standard imports
import matplotlib.pyplot as plt
import numpy as np

#setting off interactive mode, so that we can split the code among
#several cells, see:
#https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.ioff.html
plt.ioff()

# make figure and assign axis objects
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))
fig.subplots_adjust(wspace=0)

### Adding a pie chart

In [ ]:
#this magic suppresses output, without it the return value of
#the last command is printed. Unfortunately, maybe your jupyter
#does not support it. Give it a try!
#%%capture

# pie chart parameters
my_ratios = nametypes_counts
my_labels = nametypes_counts.index
my_explode = [0.3, 0]

# rotate so that first wedge is split by the x-axis
angle = -180 * ratios[0]

#standard pie plot
ax1.pie(my_ratios, autopct='%1.1f%%', startangle=angle, labels=my_labels, explode=my_explode)

### Adding a bar chart

In [ ]:
# bar chart parameters
xpos = 0
bottom = 0
ratios = fall_counts_valid
width = .2

#looping through all the available data points
for j in range(len(ratios)):
    height = ratios[j]
    ax2.bar(xpos, height, width, bottom=bottom)
    ypos = bottom + ax2.patches[j].get_height() / 2
    bottom += height
    ax2.text(xpos, ypos, "%d%%" % (ax2.patches[j].get_height() * 100), ha='center')

#final aestethics brush upss
ax2.set_title('Valid entries')
ax2.legend(fall_counts_valid.index)
ax2.axis('off')
ax2.set_xlim(- 2.5 * width, 2.5 * width)

### Adding connecting lines

We are going to use [ConnectionPatch](https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.ConnectionPatch.html) object to draw lines connecting
the pie to the bar plot.

Keep in mind that a pie slice is completely defined by four parameters:

* center
* radius (r)
* angle of the first side (theta1)
* angle of the second side (theta2)

We are going to first extract these numbers, then do some trigonometry to convert
those to coordinates:

* (x_start, y_start)
* (x_end, y_end)

for the two connectors.

In [ ]:
from matplotlib.patches import ConnectionPatch

#extracting the pie slice parameters
center = ax1.patches[0].center #this is actually two values
r      = ax1.patches[0].r
theta1 = ax1.patches[0].theta1
theta2 = ax1.patches[0].theta2

#the bar is easier: it's just defined by its height
bar_height = sum([item.get_height() for item in ax2.patches])

# top connecting line, coordinates
x_start = r * np.cos(np.pi / 180 * theta2) + center[0]
y_start = r * np.sin(np.pi / 180 * theta2) + center[1]
x_end = -width / 2
y_end = bar_height

# top connecting line, drawing
con = ConnectionPatch(xyA=(x_start, y_start), coordsA=ax1.transData,
                      xyB=(x_end, y_end)    , coordsB=ax2.transData)
con.set_color([0, 0, 0])
con.set_linewidth(2)
ax2.add_artist(con)

# bottom connecting line, coordinates
x_start = r * np.cos(np.pi / 180 * theta1) + center[0]
y_start = r * np.sin(np.pi / 180 * theta1) + center[1]
x_end = -width / 2
y_end = 0

# top connecting line, drawing
con = ConnectionPatch(xyA=(x_start, y_start), coordsA=ax1.transData,
                      xyB=(x_end, y_end)    , coordsB=ax2.transData)
con.set_color([0, 0, 0])
con.set_linewidth(2)
ax2.add_artist(con)

### Plotting the result

Let's turn the interactive mode back on and plot the result.

In [ ]:
plt.ion()
plt.show()

# Assignments

The plot works, but with a very thin slice its power is somewhat limited. Also, it would be nice to have reusable solution.

* ASSIGNMENT 1: write a function that accepts the two counts arguments and does the plot
* ASSIGNMENT 2: add a third argument to save on file (in folder "../results") instead of displaying the plot
* ASSIGNMENT 3: create a new column in the data that assigns meteorites to four categories based on their weight, and use it in the plot